# DTSC 580: Data Manipulation

## Assignment:  Halloween Candy

### Name:

## Overview

In this assignment, your job will be to clean and wrangle data from a survey of Halloween candy to prepare it for a machine learning project.

Once you have completed all the tasks in the notebook, save your notebook as `halloween_candy`, and verify that all the tests pass in CodeGrade.  

For those of you that are more comfortable with using Pandas, I have also provided extra optional data analysis questions at the end of the assignment for more practice.

## Data Set

The data set that we will be using is the 2017 Halloween Candy Hierarchy data set as discussed in this [boingboing](https://boingboing.net/2017/10/30/the-2017-halloween-candy-hiera.html) article.  You can also read more about the data in the [Science Creative Quarterly](https://www.scq.ubc.ca/so-much-candy-data-seriously/).

The following are the rating instructions from the survey:  

> Basically, consider that feeling you get when you receive this item in your Halloween haul. Does it make you really happy (JOY)? Or is it something that you automatically place in the junk pile (DESPAIR)? MEH for indifference, and you can leave blank if you have no idea what the item is.

Note that the original data set has been slightly altered from its original state, and if you wanted to perform any analysis for future projects, you would need to download the data directly from the links above.

This data is a great example of a messy data set, especially since they allowed respondents to enter text for a number of the fields. Also, note that some of the comments in the file might be considered inappropriate to some readers but cleaning this type of data is normal in a lot of data science projects.

## Note

<u>Show Work</u>

Remember that you must show your work.  Students submissions are spot checked manually throughout the term to verify that they are not hard coding the answer from looking only in the file or in CodeGrade's expected output.  If this is seen, the student's answer will be manually marked wrong and their grade will be changed to reflect this.

For example, if the answer to Q1, the mean of a specific column, is 22:
```
# correct way
Q1 = df['column_name'].mean()

# incorrect way
Q1 = 22
```

## Our End Goal

Our end goal for this project is to clean the data so that we could then create a machine learning model. We want to see if we are able to predict a person's gender based purely on their candy preferences. Although, you will not be creating a model for this assignment, only cleaning the data. The results of the models that I used after cleaning the data are provided at the end of this notebook.

## Initial Import & Exploration

In [602]:
# initial imports
import pandas as pd
import numpy as np

# Do not change this option; This allows the CodeGrade auto grading to function correctly
pd.set_option('display.max_columns', 20)
import warnings
warnings.filterwarnings("ignore")

Let's start by importing our data and creating a DataFrame called `candy`.  We need to include `encoding='iso-8859-1'` during the import because there are special characters in the data that Pandas doesn't recognize. This happens a lot when attempting to import data where the public is able to input answers, especially if there are foreign language characters included. The normal encoding for Pandas is `utf-8`, so changing the encoding allows Pandas to recognize those special characters.

Run the following code, with the encoding argument, and it should import correctly.

In [603]:
# read_csv with iso-8859-1 encoding; using latin-1 would also work here
candy_full = pd.read_csv('candy.csv', encoding='iso-8859-1')

# copy to new DF so that we can have a copy of the original import if needed
candy = candy_full.copy()

Let's take a brief look at the data by using `head()`.

In [604]:
# first five rows
candy.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


Next, run the following code to see information about the DataFrame.

In [605]:
# check info about the DataFrame
candy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: float64(4), int64(1), object(115)
memory usage: 2.3+ MB


Notice that this did not print the columns as you might be used to seeing. According to the Pandas documentation:  "If the DataFrame has more than max_cols columns, the truncated output is used. By default, the setting in pandas.options.display.max_info_columns is used."

We can make the columns display by setting the `max_cols` argument equal to the number of columns in the data set.

In [606]:
# check info, set max_cols
candy.info(max_cols=120)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 120 columns):
 #    Column                                                                                 Non-Null Count  Dtype  
---   ------                                                                                 --------------  -----  
 0    Internal ID                                                                            2479 non-null   int64  
 1    Q1: GOING OUT?                                                                         2368 non-null   object 
 2    Q2: GENDER                                                                             2437 non-null   object 
 3    Q3: AGE                                                                                2394 non-null   object 
 4    Q4: COUNTRY                                                                            2414 non-null   object 
 5    Q5: STATE, PROVINCE, COUNTY, ETC                                   

Of course, if you are just looking for the column names, you can just use a simple `for` loop.

In [607]:
# print a list of column names
for col in candy.columns:
    print(col)

Internal ID
Q1: GOING OUT?
Q2: GENDER
Q3: AGE
Q4: COUNTRY
Q5: STATE, PROVINCE, COUNTY, ETC
Q6 | 100 Grand Bar
Q6 | Anonymous brown globs that come in black and orange wrappers	(a.k.a. Mary Janes)
Q6 | Any full-sized candy bar
Q6 | Black Jacks
Q6 | Bonkers (the candy)
Q6 | Bonkers (the board game)
Q6 | Bottle Caps
Q6 | Box'o'Raisins
Q6 | Broken glow stick
Q6 | Butterfinger
Q6 | Cadbury Creme Eggs
Q6 | Candy Corn
Q6 | Candy that is clearly just the stuff given out for free at restaurants
Q6 | Caramellos
Q6 | Cash, or other forms of legal tender
Q6 | Chardonnay
Q6 | Chick-o-Sticks (we donÕt know what that is)
Q6 | Chiclets
Q6 | Coffee Crisp
Q6 | Creepy Religious comics/Chick Tracts
Q6 | Dental paraphenalia
Q6 | Dots
Q6 | Dove Bars
Q6 | Fuzzy Peaches
Q6 | Generic Brand Acetaminophen
Q6 | Glow sticks
Q6 | Goo Goo Clusters
Q6 | Good N' Plenty
Q6 | Gum from baseball cards
Q6 | Gummy Bears straight up
Q6 | Hard Candy
Q6 | Healthy Fruit
Q6 | Heath Bar
Q6 | Hershey's Dark Chocolate
Q6 | HersheyÕ

This data set is pretty messy. Your goal is now to perform the following actions to get it to the point where it can be passed to a machine learning model.

<span style='color:red'>**Note: Unless the instructions ask you to do something different, please always update the original `candy` DataFrame for the exercises below.  The automatic grading in CodeGrade will check your final DataFrame and ensure that you have performed all required data manipulations.  Also, feel free to add additional cells as needed.** </span>

<span style='color:red'>**Note 2: If you run your code through CodeGrade before you get to exercise 19, your indices might be different than what CodeGrade is expecting.  Just continue with the assignment and run your code again through CodeGrade after exercise 19 to see if that clears up the error.**</span>

## Data Cleaning

**Exercise1:** Taking a look at the column names, you may notice that some include the character `Õ`. This should instead be an apostrophe `'` mark. Rename the column names that include the `Õ` character and replace it with an apostrophe.  

Remember that you should be updating the `candy` DataFrame for the tasks listed as unless told differently.

In [608]:
### ENTER CODE HERE ###
candy = candy.rename(columns=lambda x: x.replace('Õ', "'"))

**Q1:** How many duplicated rows are there in the file? Assume that a duplicate is any row that is *exactly* the same as another one. Save this number as `Q1`.

In [609]:
### ENTER CODE HERE ###
Q1 = candy.duplicated().sum()
Q1

np.int64(17)

**Q2:** How many duplicated rows are there in the file if we were to assume that a duplicate is any row with the same `Internal ID` number as another. In other words, even if the other values are different, a row would count as a duplicate if it had the same `Internal ID` as another. Save this number as `Q2`.

In [610]:
### ENTER CODE HERE ###
Q2= candy.duplicated(subset='Internal ID').sum()
Q2

np.int64(19)

**Exercise2:** Drop any duplicates from the `candy` DataFrame.  Duplicates are to be defined as any row with the same `Internal ID` as another. Use the default setting that keeps the first record from the duplicates.

In [611]:
### ENTER CODE HERE ###
candy = candy.drop_duplicates(subset='Internal ID')

**Exercise3:** Your next task is to remove the following columns from the `candy` DataFrame as we will not use these columns for this project.  You are welcome to do further analysis on these columns but do not save your analysis in this notebook.

Remove the following columns: `Internal ID`, `Q5: STATE, PROVINCE, COUNTY, ETC`, `Q7: JOY OTHER`, `Q8: DESPAIR OTHER`, `Q9: OTHER COMMENTS`, `Unnamed: 113`, `Click Coordinates (x, y)`.

In [612]:
### ENTER CODE HERE ###
candy = candy.drop(columns=['Internal ID', 'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q7: JOY OTHER', 'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS', 'Unnamed: 113', 'Click Coordinates (x, y)'])

**Code Check:** As a check for the above exercises, the shape of your data should now be: `(2460, 113)`

In [613]:
### ENTER CODE HERE ###
candy.shape

(2460, 113)

**Exercise4:** Let's now take a look at the `Q2: GENDER` column since this will be what we are trying to predict. Take a look at the value counts for this column.

In [614]:
### ENTER CODE HERE ###
candy['Q2: GENDER'].value_counts()

,count
Q2: GENDER,
Male,1466
Female,839
I'd rather not say,83
Other,30


**Q3:** How many missing values are in the `Q2: GENDER` column? Save this as `Q3`.

In [615]:
### ENTER CODE HERE ###
Q3 = candy['Q2: GENDER'].isna().sum()
Q3

np.int64(42)

**Exercise5:** Using the `candy` DataFrame, remove all rows with a missing value in the `Q2: GENDER` column.  (This should overwrite and be saved as `candy` like you have been doing for the previous exercises.)

In [616]:
### ENTER CODE HERE ###
candy = candy.dropna(subset=['Q2: GENDER'])

**Exercise6:** For this project, we want to use binary classification, which predicts one of two classes. Because we are using a binary classification model, we will try to predict between `Male` or `Female`. Select only the rows that contain either `Male` or `Female` in the `Q2: GENDER` column.

In [617]:
### ENTER CODE HERE ###
candy=candy[candy['Q2: GENDER'].isin(['Male','Female'])]

**Code Check:** As a check for the above exercises, the shape of your data should now be: `(2305, 113)`

In [618]:
### ENTER CODE HERE ###
candy.shape

(2305, 113)

Now, let's work on filling some of the missing data.  There are easier ways to do this with the sklearn library which you will learn about more in the machine learning classes, but for now, let's try to practice our Pandas skills.

**Q4:** How many missing values are in the `Q1: GOING OUT?` column? Save this number as `Q4`.

In [619]:
### ENTER CODE HERE ###
Q4 = candy['Q1: GOING OUT?'].isna().sum()
Q4

np.int64(77)

**Exercise7:** For a future analysis question, we are interested in those that we know will *definitely* go out for Halloween.  Because of this, fill all missing values in the `Q1: GOING OUT?` column with a `No` value.

In [620]:
### ENTER CODE HERE ###
candy['Q1: GOING OUT?'].fillna('No', inplace=True)

**Code Check:** Double check your above work and look at the value counts for the `Q1: GOING OUT?` column.  Make sure that you only have "Yes" and No" values and that they add up to 2305, which is the number of rows you should have at this step in the assignment.

In [621]:
### ENTER CODE HERE ###
candy['Q1: GOING OUT?'].value_counts()

,count
Q1: GOING OUT?,
No,2007
Yes,298


**Exercise8:** To get ready for the next step, let's practice selecting all the columns: going from `Q6 | 100 Grand Bar` to `Q11: DAY`.  Save this slice as `candy_slice`.

In [622]:
### ENTER CODE HERE ###
candy_slice=candy.loc[:,'Q6 | 100 Grand Bar':'Q11: DAY']

**Exercise9:** Now that you know how to slice the data, fill any missing values in the `candy` DataFrame for those columns (going from `Q6 | 100 Grand Bar` to `Q11: DAY`) with the string `NO_ANSWER`. **Make sure you are working with the `candy` DataFrame and not the `candy_slice` DataFrame.**

In [623]:
### ENTER CODE HERE ###
candy.loc[:,'Q6 | 100 Grand Bar':'Q11: DAY'] = candy.loc[:,'Q6 | 100 Grand Bar':'Q11: DAY'].fillna('NO_ANSWER')
candy.head()

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),...,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1,No,Male,44,USA,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,NaN,1.0,NaN,NaN
2,No,Male,49,USA,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NaN,NaN,NaN,NaN
3,No,Male,40,us,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,NaN,1.0,NaN,NaN
4,No,Male,23,usa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,...,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,NaN,1.0,NaN,NaN
5,No,Male,NaN,NaN,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,JOY,DESPAIR,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NaN,1.0,NaN,NaN


**Exercise10:** For all four `Q12: Media` columns in the `candy` DataFrame, fill the missing values with `0.0`.

In [624]:
### ENTER CODE HERE ###
candy.loc[:,'Q12: MEDIA [Daily Dish]': 'Q12: MEDIA [Yahoo]']=candy.loc[:,'Q12: MEDIA [Daily Dish]': 'Q12: MEDIA [Yahoo]'].fillna(0.0)

**Code Check:** As a check for the above code, make sure that there are no missing values left for the `Q6` to `Q12` columns.  

In [625]:
### ENTER CODE HERE ###
candy.loc[:,'Q6 | 100 Grand Bar':'Q12: MEDIA [Yahoo]'].isna().sum()

,0
Q6 | 100 Grand Bar,0
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),0
Q6 | Any full-sized candy bar,0
Q6 | Black Jacks,0
Q6 | Bonkers (the candy),0
...,...
Q11: DAY,0
Q12: MEDIA [Daily Dish],0
Q12: MEDIA [Science],0
Q12: MEDIA [ESPN],0


Now, let's look at the very messy `Q4: COUNTRY` column and see what we can do about it. First, run the code below to look at the different unique values in the data.

In [626]:
# check unique values
candy['Q4: COUNTRY'].unique()

array(['USA ', 'USA', 'us', 'usa', nan, 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'United states', 'u.s.a.', '35', 'france',
       'United States of America ', 'america', 'U.S.A.', 'finland',
       'unhinged states', 'Mexico', 'Canada ', 'united states of america',
       'US of A', 'The United States', 'North Carolina ', 'Unied States',
       'Netherlands', 'germany', 'Europe', 'U S', 'u.s.', 'U.K. ',
       'Costa Rica', 'The United States of America', 'unite states',
       'U.S.', '46', 'Australia', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", '45', 'United State', '32', 'France', 'australia',
       'Can', 'Canae', 'Trumpistan', 'Ireland', 'United Sates', 'Korea',
       'California', 'Unites States', 'Japan', 'USa', 'South africa',
       'I pretend to be from Canada, but I am

**Code Check:** As a check for the Country column, check to see how many unique values are in the data.  You should have `115` different unique values for the `Q4: COUNTRY` column.  If you have less or more than this number, double check your work above.

In [627]:
# check the Q4: COUNTRY number of unique values
candy['Q4: COUNTRY'].nunique()

115

We want to clean up this data to only include four areas: USA, Canada, Europe (the continent, not necessarily the European Union), and Other.

There are different ways to do this, but I would suggest that you look at the way we handled the `property_type` column in the `vienna` data set and the code in the `amenities_to_columns()` function in the module notebook.  These might be a little harder than those examples but they should give you a good baseline approach.  

You could use `replace()` for this step, and it is fine if you ultimately decide to do this, but I would suggest that you come up with a solution similar to what was shown in the `vienna` data cleaning notebook.  This method would be much more robust if you had many more values in your data.

I suggest the following order for this section to make it easier:
- Fill in all missing values with `Other`
- Code Australia as `Other` (doing this step will help when trying to use `us` in the next step if you use string methods)
- Combine all USA entries together as `USA`
- Combine Canadian entries as `CA`
- Combine European entries as `EU`
- Everything else gets coded as `Other`

**Exercise11:** Fill the missing values in the `Q4: COUNTRY` column with `Other`.

In [628]:
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].fillna('Other', inplace=True)

**Code Check:** Double check that there are no missing values in the `Q4: COUNTRY` column.  Also, double check the unique values to make sure that "Other" was added.  This should mean that you now have `116` unique values for this column.

In [629]:
# check missing Q4 values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].isna().sum()

np.int64(0)

In [630]:
# check unique values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].nunique()

116

**Exercise12:** Combine all Australia entries into `Other`.  Watch out for capitalization issues.  You should have `114` unique values after this step.

In [631]:
### ENTER CODE HERE ###
candy['Q4: COUNTRY'] = candy['Q4: COUNTRY'].replace({'Australia': 'Other','australia':'Other'})

In [632]:
# check number of unique values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].nunique()

114

**Exercise13:** Combine all United States entries together into `USA`.  These would include the following:
```
'USA ', 'USA', 'us', 'usa', 'Us', 'US', 'Murica', 'United States', 'united states', 'Usa', 'United States ', 'United staes', 'United States of America', 'United states', 'u.s.a.', 'United States of America ', 'america', 'U.S.A.', 'unhinged states', 'united states of america', 'US of A', 'The United States', 'North Carolina ', 'Unied States', 'U S', 'u.s.', 'The United States of America', 'unite states','U.S.', 'USA? Hard to tell anymore..', "'merica", 'United State', 'United Sates', 'California', 'Unites States', 'USa', 'I pretend to be from Canada, but I am really from the United States.', 'Usa ', 'United Stated', 'New Jersey', 'United ststes', 'America', 'United Statss', 'murrika', 'USA! USA! USA!', 'USAA', 'united States ', 'N. America', 'USSA', 'U.S. ', 'u s a', 'United Statea', 'united ststes', 'USA USA USA!!!!'
```

**Careful:** Some students try to use regex for this step with a string method.  Be careful as there is one response with the term "subscribe" in it.  If you create your regex incorrectly, your use of "u.s." could mistakenly pick up this record because of the "ubs" contained in it and code it as "USA".

In [633]:
### ENTER CODE HERE ###
candy['Q4: COUNTRY'] = candy['Q4: COUNTRY'].replace(['USA ', 'USA', 'us', 'usa','USA! USA! USA!', 'Us', 'US', 'Murica', 'United States', 'united states', 'Usa', 'United States ', 'United staes', 'United States of America', 'United states', 'u.s.a.', 'United States of America ', 'america', 'U.S.A.', 'unhinged states', 'united states of america', 'US of A', 'The United States', 'North Carolina ', 'Unied States', 'U S', 'u.s.', 'The United States of America', 'unite states','U.S.', 'USA? Hard to tell anymore..', "'merica", 'United State', 'United Sates', 'California', 'Unites States', 'USa', 'I pretend to be from Canada, but I am really from the United States.', 'Usa ', 'United Stated', 'New Jersey', 'United ststes', 'America', 'United Statss', 'murrika', 'USA! USA! USA!!!!', 'USAA', 'united States ', 'N. America', 'USSA', 'U.S. ', 'u s a', 'United Statea', 'united ststes', 'USA USA USA!!!!'], 'USA')

**Code Check:** You should be merging the above values together into 1 (`USA`) and be left with 61 unique values after this step (including the `USA` value).

In [634]:
# check unique values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].unique()

array(['USA', 'Other', 'canada', 'Canada', 'uk', 'United Kingdom', 'UAE',
       'England', 'UK', 'canada ', '35', 'france', 'finland', 'Mexico',
       'Canada ', 'Netherlands', 'germany', 'Europe', 'U.K. ',
       'Costa Rica', '46', 'Greece', '45', '32', 'France', 'Can', 'Canae',
       'Trumpistan', 'Ireland', 'Korea', 'Japan', 'South africa', 'Uk',
       'Germany', 'Canada`', 'Scotland', 'UK ', 'Denmark', 'France ',
       'Switzerland', 'UD', 'Scotland ', 'South Korea', 'CANADA',
       'Indonesia', 'The Netherlands', 'endland', 'soviet canuckistan',
       'Singapore', 'China', 'Taiwan', 'Ireland ', 'hong kong', 'spain',
       'Sweden', 'Hong Kong', 'Narnia', 'subscribe to dm4uz3 on youtube',
       'United kingdom', "I don't know anymore", 'Fear and Loathing'],
      dtype=object)

**Exercise14:** Combine the Canadian entries (both upper and lower case) and label them as `CA`. Be careful as there are extra spaces, characters, and misspellings (Can, Canae).

These values include:
```
'canada', 'Canada', 'canada ', 'Canada ', 'Can', 'Canae', 'Canada`', 'CANADA'
```

In [635]:
### ENTER CODE HERE ###
candy['Q4: COUNTRY'] = candy['Q4: COUNTRY'].replace(['canada', 'Canada', 'canada ', 'Canada ', 'Can', 'Canae', 'Canada`', 'CANADA'], 'CA')

**Code Check:** You should be merging 8 values together into 1 (`CA`) and be left with 54 unique values after this step (including the `CA` value).

In [636]:
# check unique values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].nunique()

54

**Exercise15:** Combine the European entries and label them as `EU`. Again, we are looking at the continent of Europe and not necessarily the countries that are a part of the European Union.  

These values include:
```
'uk', 'United Kingdom', 'England', 'UK', 'france', 'finland', 'Netherlands', 'germany', 'Europe', 'U.K. ', 'Greece', 'France', 'Ireland', 'Uk', 'Germany', 'Scotland', 'UK ', 'Denmark', 'France ', 'Switzerland', 'Scotland ', 'The Netherlands', 'Ireland ', 'spain', 'Sweden', 'United kingdom'
```

In [637]:
### ENTER CODE HERE ###
candy['Q4: COUNTRY'] = candy['Q4: COUNTRY'].replace(['uk', 'United Kingdom', 'England', 'UK', 'france', 'finland', 'Netherlands', 'germany', 'Europe', 'U.K. ', 'Greece', 'France', 'Ireland', 'Uk', 'Germany', 'Scotland', 'UK ', 'Denmark', 'France ', 'Switzerland', 'Scotland ', 'The Netherlands', 'Ireland ', 'spain', 'Sweden', 'United kingdom'], 'EU')

**Code Check:** You should be merging 26 entries together and be left with 29 unique values after this step (including the `EU` value).

In [638]:
# check unique values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].unique()

array(['USA', 'Other', 'CA', 'EU', 'UAE', '35', 'Mexico', 'Costa Rica',
       '46', '45', '32', 'Trumpistan', 'Korea', 'Japan', 'South africa',
       'UD', 'South Korea', 'Indonesia', 'endland', 'soviet canuckistan',
       'Singapore', 'China', 'Taiwan', 'hong kong', 'Hong Kong', 'Narnia',
       'subscribe to dm4uz3 on youtube', "I don't know anymore",
       'Fear and Loathing'], dtype=object)

**Exercise16:** Finally, combine the other entries and label them as `Other`.

In [639]:
### ENTER CODE HERE ###
candy.loc[~candy['Q4: COUNTRY'].isin(['USA', 'CA', 'EU']), 'Q4: COUNTRY'] = 'Other'

**Code Check:** Double check that you only have four unique values in the `Q4: COUNTRY` column: `USA`, `Other`, `CA`, and `EU`

In [640]:
# check values
### ENTER CODE HERE ###
candy['Q4: COUNTRY'].unique()

array(['USA', 'Other', 'CA', 'EU'], dtype=object)

**Q5:** To double check that everything was coded correctly, save the value counts of the `Q4: COUNTRY` column as `Q5`.  You can check this once you run your CodeGrade check.

In [641]:
### ENTER CODE HERE ###
Q5 = candy['Q4: COUNTRY'].value_counts()
Q5

,count
Q4: COUNTRY,
USA,1955
CA,216
EU,73
Other,61


We now want to look at the `Q3: AGE` column. Let's look at all the unique values.

In [642]:
# check unique age values
### ENTER CODE HERE ###
candy['Q3: AGE'].unique()

array(['44', '49', '40', '23', nan, '53', '33', '43', '56', '64', '37',
       '48', '54', '36', '45', '25', '34', '35', '38', '58', '50', '47',
       '16', '52', '63', '65', '41', '27', '31', '59', '61', '46', '42',
       '62', '29', '39', '32', '28', '69', '67', '30', '22', '51', '70',
       '24', '19', 'Old enough', '57', '60', '66', '12', 'Many', '55',
       '72', '?', '21', '11', 'no', '9', '68', '20', '6', '10', '71',
       '13', '26', '45-55', '7', '39.4', '74', '18', 'older than dirt',
       '17', '15', '8', '75', '5u', 'Enough', 'Over 50', '90', '76',
       'sixty-nine', 'ancient', '77', 'OLD', 'old', '73', '70 1/2', '14',
       'MY NAME JEFF', '4', '59 on the day after Halloween', 'old enough',
       'your mom', 'I can remember when Java was a cool new language',
       '60+'], dtype=object)

Again, this is a pretty messy column of data. This is a good example of why those that create online surveys shouldn't allow the individual to just put any value into the field. But it is now our job to clean this up.

**Exercise17:** Your task is to put these values into the following categorical bins: `unknown`, `17 and under`, `18-25`, `26-35`, `36-45`, `46-55`, and `56+`.

- The category labels should exactly match the above.
- Missing values should be replaced with the `unknown` category
- To make things easier and avoid ambiguity, let's say that any value with text, even if we could determine the age, will be binned with the `unknown` category. For example: `sixty-nine` should be coded as `unknown`, `45-55` should be coded as `unknown`, `59 on the day after Halloween` should be coded as `unknown`, etc.
- Ensure that the category labels are unordered but reorder the categories so that 'unknown' is listed in the first position. This is not really needed but will help us grade your assignment. The categories should be listed as follows: `Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')`

First, we will replace any non-numeric value (those with text as mentioned above) with a missing value.  This will allow you to turn the other values into floats so that you can bin them. Just don't forget to code the missing values as `unknown` when you are done.  To replace the non-numeric values, run the following code:

In [643]:
### DO NOT CHANGE THIS CODE - RUN AS IS ###
### This code replaces non-numeric values with a missing value to make your next step easier ###

# create True/False index
age_index = candy['Q3: AGE'].str.isnumeric()

# for the index, fill missing values with False
age_index = age_index.fillna(False)

# select Age column for only those False values from index and code as missing
candy.loc[~age_index, 'Q3: AGE'] = np.nan

Now, you can finish exercise 17 to bin the ages and write the code below:

In [644]:
### ENTER REST OF CODE HERE ###
candy['Q3: AGE'] = pd.to_numeric(candy['Q3: AGE'], errors='coerce')
candy['Q3: AGE']
bins = [0, 17, 25, 35, 45, 55, 100]
labels = ['17 and under', '18-25', '26-35', '36-45', '46-55','56+']
candy['Q3: AGE'] = pd.cut(x=candy['Q3: AGE'], bins=bins, labels=labels)
candy['Q3: AGE'] = candy['Q3: AGE'].cat.add_categories('unknown')
candy['Q3: AGE'] = candy['Q3: AGE'].fillna('unknown')
new_order = ['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+']
candy['Q3: AGE'] = candy['Q3: AGE'].cat.reorder_categories(new_order)
candy['Q3: AGE']

,Q3: AGE
1,36-45
2,46-55
3,36-45
4,18-25
5,unknown
...,...
2474,18-25
2475,26-35
2476,26-35
2477,56+


**Exercise18:** Double check yourself by checking the categories for the `Q3: AGE` column. It should output: `Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')`

In [645]:
# double check categories
### ENTER CODE HERE ###
candy['Q3: AGE'].cat.categories

Index(['unknown', '17 and under', '18-25', '26-35', '36-45', '46-55', '56+'], dtype='object')

**Code Check:** To double check your above binning worked correctly, your value counts (sorted by the index) should be as follows:

```
unknown: 60
17 and under: 49
18-25: 85
26-35: 520
36-45: 768
46-55: 525
56+: 298
```

In [646]:
### ENTER CODE HERE ###
candy['Q3: AGE'].value_counts().sort_index()

,count
Q3: AGE,
unknown,60
17 and under,49
18-25,85
26-35,520
36-45,768
46-55,525
56+,298


You can also double check some of your work up to this point by making sure that there are no missing values in the data set anymore.

**Code Check:** Check to see if there are any missing values in the data set. Your output should show `0`.

In [647]:
### ENTER CODE HERE ###
candy.isna().sum()

,0
Q1: GOING OUT?,0
Q2: GENDER,0
Q3: AGE,0
Q4: COUNTRY,0
Q6 | 100 Grand Bar,0
...,...
Q11: DAY,0
Q12: MEDIA [Daily Dish],0
Q12: MEDIA [Science],0
Q12: MEDIA [ESPN],0


**Exercise19:** Before you move on to the next section, reset the index for `candy` ensuring that it goes from 0 to n-1.  

In [660]:
### ENTER CODE HERE ###
candy = candy.reset_index(drop=True)

## Feature Engineering

Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models.  In this section, we will create a new column called "net_feelies" (calculated by the authors as the total joy count minus the total despair count).

First, let's narrow down our data to make working with it easier.

**Exercise20:** Select only the Q6 candy columns (`Q6 | 100 Grand Bar` through `Q6 | York Peppermint Patties`) in the data set and save this as a new DataFrame called `candy_reduced`.

In [664]:
### ENTER CODE HERE ###
candy_reduced = candy.loc[:,'Q6 | 100 Grand Bar':'Q6 | York Peppermint Patties']
candy_reduced.head()

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
0,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR
1,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER
2,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR
3,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY
4,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY


Next, we will create two Series, one with JOY counts and one with DESPAIR counts to add to our `candy_reduced` data.

**Exercise21:** Create a Series called `joy_count` that lists total counts for JOY for each column, making sure to keep it in the same order as the columns in the `candy_reduced` DataFrame. Hint: A simple way to do this is to filter the entire DataFrame for any `JOY` values and then use `count()`. See this [stackoverflow question](https://stackoverflow.com/questions/63103090/how-do-i-count-specific-values-across-multiple-columns-in-pandas) and answers.

In [666]:
### ENTER CODE HERE ###
joy_count = candy_reduced[candy_reduced == 'JOY'].count()
joy_count


,0
Q6 | 100 Grand Bar,828
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),165
Q6 | Any full-sized candy bar,1477
Q6 | Black Jacks,87
Q6 | Bonkers (the candy),109
...,...
Q6 | Vicodin,674
Q6 | Whatchamacallit Bars,802
Q6 | White Bread,43
Q6 | Whole Wheat anything,110


**Exercise22:** Same as above except you will create a Series called `despair_count` that lists the total counts for DESPAIR for each column.

In [668]:
### ENTER CODE HERE ###
despair_count =  candy_reduced[candy_reduced == 'DESPAIR'].count()
despair_count

,0
Q6 | 100 Grand Bar,80
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),1017
Q6 | Any full-sized candy bar,15
Q6 | Black Jacks,754
Q6 | Bonkers (the candy),467
...,...
Q6 | Vicodin,679
Q6 | Whatchamacallit Bars,273
Q6 | White Bread,1376
Q6 | Whole Wheat anything,1233


**Exercise23:** Take the transpose of the `candy_reduced` DataFrame and save this transposed data as `candy_reduced_transpose`.

In [670]:
### ENTER CODE HERE ###
candy_reduced_transpose = candy_reduced.transpose()
candy_reduced_transpose

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,2304
Q6 | 100 Grand Bar,MEH,NO_ANSWER,MEH,JOY,JOY,NO_ANSWER,JOY,JOY,MEH,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,JOY,MEH,JOY,JOY,MEH,MEH,NO_ANSWER,DESPAIR
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR
Q6 | Any full-sized candy bar,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,JOY,JOY,JOY,MEH,JOY,JOY,NO_ANSWER,JOY
Q6 | Black Jacks,MEH,NO_ANSWER,MEH,DESPAIR,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,NO_ANSWER,DESPAIR
Q6 | Bonkers (the candy),DESPAIR,NO_ANSWER,MEH,MEH,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,MEH,MEH,MEH,DESPAIR,NO_ANSWER,MEH,NO_ANSWER,DESPAIR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q6 | Vicodin,DESPAIR,NO_ANSWER,JOY,JOY,DESPAIR,NO_ANSWER,MEH,DESPAIR,DESPAIR,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,JOY,DESPAIR,MEH,JOY,NO_ANSWER,JOY,NO_ANSWER,JOY
Q6 | Whatchamacallit Bars,DESPAIR,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,MEH,NO_ANSWER,JOY,DESPAIR,JOY,MEH,NO_ANSWER,DESPAIR
Q6 | White Bread,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,NO_ANSWER,MEH
Q6 | Whole Wheat anything,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,MEH,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,NO_ANSWER,DESPAIR


**Exercise24:** Add a new column called "joy_count" using the `joy_count` Series above and a new column called 'despair_count" using the `despair_count` Series above to the `candy_reduced_transpose` DataFrame.

In [673]:
### ENTER CODE HERE ###
candy_reduced_transpose['joy_count'] = joy_count
candy_reduced_transpose['despair_count'] = despair_count
candy_reduced_transpose

,0,1,2,3,4,5,6,7,8,9,...,2297,2298,2299,2300,2301,2302,2303,2304,joy_count,despair_count
Q6 | 100 Grand Bar,MEH,NO_ANSWER,MEH,JOY,JOY,NO_ANSWER,JOY,JOY,MEH,NO_ANSWER,...,JOY,MEH,JOY,JOY,MEH,MEH,NO_ANSWER,DESPAIR,828,80
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,165,1017
Q6 | Any full-sized candy bar,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,...,JOY,JOY,JOY,MEH,JOY,JOY,NO_ANSWER,JOY,1477,15
Q6 | Black Jacks,MEH,NO_ANSWER,MEH,DESPAIR,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,NO_ANSWER,DESPAIR,87,754
Q6 | Bonkers (the candy),DESPAIR,NO_ANSWER,MEH,MEH,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,MEH,MEH,MEH,DESPAIR,NO_ANSWER,MEH,NO_ANSWER,DESPAIR,109,467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q6 | Vicodin,DESPAIR,NO_ANSWER,JOY,JOY,DESPAIR,NO_ANSWER,MEH,DESPAIR,DESPAIR,NO_ANSWER,...,JOY,DESPAIR,MEH,JOY,NO_ANSWER,JOY,NO_ANSWER,JOY,674,679
Q6 | Whatchamacallit Bars,DESPAIR,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,MEH,NO_ANSWER,JOY,DESPAIR,JOY,MEH,NO_ANSWER,DESPAIR,802,273
Q6 | White Bread,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,NO_ANSWER,MEH,43,1376
Q6 | Whole Wheat anything,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,MEH,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,NO_ANSWER,DESPAIR,110,1233


**Exercise25:** Add a new column to the `candy_reduced_transpose` DataFrame called "net_feelies" that takes the `joy_count` column and subtracts the `despair_count` column.

In [674]:
### ENTER CODE HERE ###
candy_reduced_transpose['net_feelies'] = candy_reduced_transpose['joy_count'] - candy_reduced_transpose['despair_count']
candy_reduced_transpose

,0,1,2,3,4,5,6,7,8,9,...,2298,2299,2300,2301,2302,2303,2304,joy_count,despair_count,net_feelies
Q6 | 100 Grand Bar,MEH,NO_ANSWER,MEH,JOY,JOY,NO_ANSWER,JOY,JOY,MEH,NO_ANSWER,...,MEH,JOY,JOY,MEH,MEH,NO_ANSWER,DESPAIR,828,80,748
Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,165,1017,-852
Q6 | Any full-sized candy bar,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,...,JOY,JOY,MEH,JOY,JOY,NO_ANSWER,JOY,1477,15,1462
Q6 | Black Jacks,MEH,NO_ANSWER,MEH,DESPAIR,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,NO_ANSWER,DESPAIR,87,754,-667
Q6 | Bonkers (the candy),DESPAIR,NO_ANSWER,MEH,MEH,NO_ANSWER,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,MEH,MEH,DESPAIR,NO_ANSWER,MEH,NO_ANSWER,DESPAIR,109,467,-358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q6 | Vicodin,DESPAIR,NO_ANSWER,JOY,JOY,DESPAIR,NO_ANSWER,MEH,DESPAIR,DESPAIR,NO_ANSWER,...,DESPAIR,MEH,JOY,NO_ANSWER,JOY,NO_ANSWER,JOY,674,679,-5
Q6 | Whatchamacallit Bars,DESPAIR,NO_ANSWER,JOY,JOY,JOY,NO_ANSWER,DESPAIR,MEH,MEH,NO_ANSWER,...,NO_ANSWER,JOY,DESPAIR,JOY,MEH,NO_ANSWER,DESPAIR,802,273,529
Q6 | White Bread,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,NO_ANSWER,MEH,43,1376,-1333
Q6 | Whole Wheat anything,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,DESPAIR,DESPAIR,DESPAIR,NO_ANSWER,...,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,NO_ANSWER,DESPAIR,110,1233,-1123


**Exercise26:** Select only the `joy_count`, `despair_count`, and `net_feelies` columns from the `candy_reduced_transpose` DataFrame. Sort this DataFrame in descending order by `net_feelies` and save this as `candy_net_sorted`.

In [675]:
### ENTER CODE HERE ###
candy_net_sorted = candy_reduced_transpose[['joy_count', 'despair_count', 'net_feelies']].sort_values(by='net_feelies', ascending=False)
candy_net_sorted

,joy_count,despair_count,net_feelies
Q6 | Any full-sized candy bar,1477,15,1462
Q6 | Reese's Peanut Butter Cups,1416,88,1328
Q6 | Kit Kat,1367,47,1320
"Q6 | Cash, or other forms of legal tender",1363,63,1300
Q6 | Twix,1339,67,1272
...,...,...,...
Q6 | Dental paraphenalia,84,1356,-1272
Q6 | Real Housewives of Orange County Season 9 Blue-Ray,86,1398,-1312
Q6 | White Bread,43,1376,-1333
Q6 | Gum from baseball cards,43,1386,-1343


## Encoding

We now want to get the `candy` DataFrame ready to run a machine learning algorthim to determine if we could predict a person's gender based on what candy they prefer.

You will learn more about this in the machine learning classes, but some algorithms work exclusively with numeric values. We will now turn all of our values into numeric values.  There are easier ways to do this with sklearn, which you will study in later courses, but we will use Pandas to perform these exercises for further practice.

**Exercise27:** For grading purposes, we want to leave the `candy` DataFrame as is. Make a copy of the `candy` DataFrame and save this new DataFrame as `candy_encode`.

In [676]:
### ENTER CODE HERE ###
candy_encode = candy.copy()

**Exercise28:** For the `candy_encode` DataFrame, replace any `Female` values with `0` and any `Male` values with `1`.

In [678]:
### ENTER CODE HERE ###
candy_encode['Q2: GENDER'] = candy_encode['Q2: GENDER'].replace({'Female': 0, 'Male': 1})
candy_encode['Q2: GENDER']

,Q2: GENDER
0,1
1,1
2,1
3,1
4,1
...,...
2300,1
2301,0
2302,0
2303,1


**Exercise29:** Again, you will learn more about this later, but we need to separate the column that we want to predict (called the response) and the columns that we will use to make the predictions (called the features).  **For both of the items below, make sure that the index is reset and goes from 0 to n-1.**

- Select only the `Q2: GENDER` column from `candy_encode` and save this as `candy_response`.  **Note: This should be a Series.**
- Drop the following columns from the `candy_encode` DataFrame: `Q2: GENDER`,`Q1: GOING OUT?`,`Q3: AGE`,`Q4: COUNTRY`,`Q10: DRESS`,`Q11: DAY`, `Q12: MEDIA [Daily Dish]`,`Q12: MEDIA [Science]`,`Q12: MEDIA [ESPN]`,`Q12: MEDIA [Yahoo]`.  Save the remaining columns as `candy_features`.

In [680]:
### ENTER CODE HERE ###
candy_response = candy_encode['Q2: GENDER']
candy_response
candy_features = candy_encode.drop(columns=['Q2: GENDER','Q1: GOING OUT?','Q3: AGE','Q4: COUNTRY','Q10: DRESS','Q11: DAY', 'Q12: MEDIA [Daily Dish]','Q12: MEDIA [Science]','Q12: MEDIA [ESPN]','Q12: MEDIA [Yahoo]'])
candy_features

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
0,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,...,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR
1,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER
2,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR
3,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,...,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY
4,JOY,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,MEH,MEH,DESPAIR,JOY,...,JOY,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,...,MEH,MEH,JOY,JOY,MEH,JOY,DESPAIR,MEH,DESPAIR,MEH
2301,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,DESPAIR,DESPAIR,JOY,...,MEH,MEH,DESPAIR,JOY,NO_ANSWER,NO_ANSWER,JOY,DESPAIR,MEH,JOY
2302,MEH,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,MEH,MEH,DESPAIR,...,JOY,JOY,MEH,MEH,MEH,JOY,MEH,DESPAIR,DESPAIR,MEH
2303,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,...,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER,NO_ANSWER


**Exercise30:** Use Panda's `get_dummies()` to encode the `candy_features` data, making sure to set `drop_first=True` and `dtype=int`. Save this as `candy_features_encoded`.

In [681]:
### ENTER CODE HERE ###
candy_features_encoded = pd.get_dummies(candy_features, drop_first=True, dtype=int)
candy_features_encoded

,Q6 | 100 Grand Bar_JOY,Q6 | 100 Grand Bar_MEH,Q6 | 100 Grand Bar_NO_ANSWER,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_JOY,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_MEH,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)_NO_ANSWER,Q6 | Any full-sized candy bar_JOY,Q6 | Any full-sized candy bar_MEH,Q6 | Any full-sized candy bar_NO_ANSWER,Q6 | Black Jacks_JOY,...,Q6 | Whatchamacallit Bars_NO_ANSWER,Q6 | White Bread_JOY,Q6 | White Bread_MEH,Q6 | White Bread_NO_ANSWER,Q6 | Whole Wheat anything_JOY,Q6 | Whole Wheat anything_MEH,Q6 | Whole Wheat anything_NO_ANSWER,Q6 | York Peppermint Patties_JOY,Q6 | York Peppermint Patties_MEH,Q6 | York Peppermint Patties_NO_ANSWER
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
2301,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2302,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2303,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1


**Code Check:** Make sure that the `candy`, `candy_features`, `candy_response` and `candy_features_encoded` have an index that goes from 0 to n-1 or your final CodeGrade tests will not pass.

## Final Analysis

Great work! You have now cleaned your data and prepared it to be passed to a machine learning model.  

I created models using Random Forest, Logistic Regression, and XGBoost algorithms, and they all returned around 70% accuracy rates. However, the other accuracy metrics (that you will learn about more in the machine learning classes) didn't look as good. Given the metrics that were calculated, I would say that based only on this data, using candy preference is not that great of an indicator of someone's gender.

**Next Steps:**  
- Submit your notebook to CodeGrade to check your final DataFrames.  
- For those of you that would like more practice using Pandas, try answering these optional questions below.

## Optional Data Analysis Questions

- How many rows were in the original, uncleaned data that you imported? How many rows are in the cleaned data? How many did we end up removing from the data set?
- What percentage of respondents are planning to go out trick-or-treating?
- What percentage of respondents 17 and younger are planning to go out for trick-or-treating?
- What are the total value counts and the normalized percentages of the age categories from the age column?
- What are the total counts and percentages for the gender column?
- What are the breakdown of counts for the country column?
- How many total respondents voted joy in candy corn and how many voted despair? Did more people vote joy or despair for candy corn?
- How many people voted joy in Reese's Peanut Butter Cups? In Snickers? Did more people vote joy for Reese's Peanut Butter Cups or for Snickers?
- How many people voted joy in Twix? In Kit Kats? Did more people vote joy for Twix or for Kit Kats?
- How many people voted joy in white bread? For whole wheat items? Did more people vote joy for white bread or whole wheat items?
- How many people voted joy for Bonkers the board game? For Bonkers the candy? Did more people vote joy for the board game or for the candy?
- How many people voted joy for a box of raisins? For the Blue-Ray DVD of the Real Housewives of Orange County Season 9? Did more people vote joy for a box of raisins or for the DVD?
- What is the favorite day of the week for the respondents (both by total counts and percentages)?
- Do more respondents see 'white and gold' or 'blue and black' for the [color of the dress](https://en.wikipedia.org/wiki/The_dress) (both total counts and percentages)?
- For those respondents that clicked on the media link (listed as Q12 columns on the survey), which link did they click on the most?
- How many males voted joy for receiving a copy of the Real Housewives of Orange County Season 9 DVD for Halloween? Females? Those 17 or younger?
- The authors tried a funny way to determine a respondent's political leaning. Note this was purely a joke and not meant to be scientific. How many total respondents voted joy in the following: Blue M&M's, Red M&M's, Green Party M&M's, Independent M&M's, and Abstained from M&M'ing?
- Determine what candy/item from the `candy_reduced` DataFrame has the most number of JOY votes and the least number of JOY votes.
- What candy/item has the most DESPAIR votes?
- What candy/item has the most MEH votes?
- What candy/item did the most people not recognize or have an opinion on? (According to the survey, the respondents were asked to leave a question blank if they did not know the item)